### Project 3

Refer to the project writeup [here](https://rutgers.app.box.com/s/k5jsawk0jwsprkcha0z2n2tli3pv53l7)


In [1]:
# next step is to make an actual testing set
    #should make sure the testing set doesn't include data from the training set
#what happens if we use sigmoid function to predict the actual value of the diagram

import math
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from matplotlib.colors import ListedColormap, BoundaryNorm


In [2]:
class diagram:
    def __init__(self, data):
        self.data = data #this will be a 20x20 matrix

    @classmethod
    def createDiagram(cls):
        dgrm = cls(np.zeros((20,20,5)))
        return dgrm
    
    #takes in a color as an integer [0-4] and returns the one hot encoded representation of it
    def oneHotEncodeColor(self, color):
        match color:
            case 0: return (0,0,0,0,1) #white
            case 1: return (0,0,0,1,0) #red
            case 2: return (0,0,1,0,0) #blue
            case 3: return (0,1,0,0,0) #yellow
            case 4: return (1,0,0,0,0) #green

    def layWires(self):
        possibleRows = [i for i in range(self.data.shape[0])] 
        possibleColumns = [i for i in range(self.data.shape[1])]

        colors = [1,2,3,4] #1 is red, 2 is blue, 3 is yellow, 4 is green
        layRowFirst = random.randrange(2) #0 indicates false, 1 indicates true
        dangerous = 0 #boolean to keep track of whether the diagram is dangerous. #0 means not dangerous, 1 meeans it is dangerous
        thirdWireLaid = 0 #this will hold the color of the 3rd wire laid down
        for i in range(4):
            color = colors.pop(random.randrange(len(colors)))
            
            #if the wire to be laid is yellow and the red wire is already laid, mark diagram as dangerous
            if color == 3 and (1 not in colors): dangerous = 1
            
            if i==2: thirdWireLaid = color
            if i%2 == 0:
                if layRowFirst == 1:
                    row = possibleRows.pop(random.randrange(len(possibleRows)))
                    for j in range(self.data.shape[0]): self.data[row][j] = self.oneHotEncodeColor(color)
                else:
                    col = possibleColumns.pop(random.randrange(len(possibleColumns)))
                    for j in range(self.data.shape[1]): self.data[j][col] = self.oneHotEncodeColor(color)
            else:
                if layRowFirst == 1:
                    col = possibleColumns.pop(random.randrange(len(possibleColumns)))
                    for j in range(self.data.shape[1]): self.data[j][col] = self.oneHotEncodeColor(color)
                else:
                    row = possibleRows.pop(random.randrange(len(possibleRows)))
                    for j in range(self.data.shape[0]): self.data[row][j] = self.oneHotEncodeColor(color)

        return (dangerous,self.oneHotEncodeColor(0)) if dangerous == 0 else (dangerous,self.oneHotEncodeColor(thirdWireLaid))


    def showDiagram(self):
        colors = ['white', 'red', 'blue', 'yellow','green']
        custom_map = ListedColormap(colors)
        bounds = [-.1, .5, 1.5, 2.5, 3.5, 4.5]
        norm = BoundaryNorm(bounds, custom_map.N)
        plt.imshow(self.data, cmap = custom_map, norm=norm)
        
        #Ensure shows plot with increments of 1
        plt.xticks(np.arange(-0.5, self.data.shape[1], 1), labels=[])
        plt.yticks(np.arange(-0.5, self.data.shape[0], 1), labels=[])

        #Show gridlines 
        plt.grid(True)

        #Show colorbar
        plt.colorbar()
        plt.show()


In [3]:
#testing if my one hot encoding is even working
# dgrm = diagram.createDiagram()
# danger = dgrm.layWires()
#dgrm.showDiagram()

data = np.load("Task1_Trainset2000/data.npy")
labels = np.load("Task1_Trainset2000/labels.npy")

print(labels)

# print(danger)
# print(dgrm.data)

[[0 0]
 [0 0]
 [1 4]
 ...
 [1 3]
 [1 3]
 [1 1]]


In [16]:
#code to generate data
def makeTask1Sets(N, dest):
    matrices = []
    labels = []
    for _ in range(int(N)):
        dgrm = diagram.createDiagram()
        danger = np.array(dgrm.layWires())
        matrices.append(dgrm.data.flatten())
        labels.append(danger)
    matrices = np.array(matrices)
    labels = np.array(labels)

    # In case the necessary directory hasn't been made yet
    #os.makedirs(dest, exist_ok=True)
    np.save(arr=matrices,file=f'{dest}/data.npy',allow_pickle=True)
    np.save(arr=labels,file=f'{dest}/labels.npy',allow_pickle=True)

def makeTask2Sets(N, dest):
    matrices = []
    labels = []

    count=0
    while(count<int(N)):
        dgrm = diagram.createDiagram()
        wireToCut = np.array((dgrm.layWires())[1])
        #print(f"my wire to cut: {wireToCut}")
        if not np.array_equal(wireToCut,(0,0,0,0,1)):
            matrices.append(dgrm.data.flatten())
            labels.append(wireToCut)
            count+=1

    matrices = np.array(matrices)
    labels = np.array(labels)

    # In case the necessary directory hasn't been made yet
    #os.makedirs(dest, exist_ok=True)
    np.save(arr=matrices,file=f'{dest}/data.npy',allow_pickle=True)
    np.save(arr=labels,file=f'{dest}/labels.npy',allow_pickle=True)

makeTask2Sets(5000,"OHE_Task2_Testset5000")

#data_t = np.load("OHE_Task2_Trainset5/data.npy")
# labels_t = np.load("OHE_Task2_Trainset2000/labels.npy")

# print(labels_t)

# for i in range(5):
#     print(data_t[i])


In [5]:
np.random.seed(42)
def SGD(w, x, y, alpha):
    # compute estimated output y'
    y_prime = sigmoid(float(np.dot(x,w)))
    # compute Loss of y' compared to y where y is the actual/correct output
    cur_loss = loss(y, y_prime) #+ (1e-10 * np.sum(np.abs(w))) # l2 reg can be changed for the model
    # SGD with new weights = old weights - alpha (sig(w * x) - y)xi) 
    w -= alpha * (y_prime - y) * x
    return cur_loss

def sigmoid(z):
    return (1 / (1 + np.exp(-z)))

def avg_loss(data, w, labels):
    losses = [loss(sigmoid(np.dot(x, w)), y) for x, y in zip(data, labels[:, 0])]
    return np.average(losses)

def loss(actual_y, y_prime):
    epsilon = 1e-15
    return -actual_y * np.log(y_prime+epsilon) - (1-actual_y) * np.log(1-y_prime+epsilon)

def add_polynomial_features(X, degree):
    """Add polynomial features to the input data."""
    features = [X]
    for d in range(2, degree + 1):
        features.append(X**d)
    return np.column_stack(features)


In [6]:
def accuracy(data, labels, weights):
    outs = np.dot(data, weights)
    binary = (outs >= 0.5).astype(int)

    samples = data.shape[0]
    correct = sum(binary == labels[:,0])
    return correct/samples


In [7]:
def feed_forward(data,labels, num_iters = 10000):
    data = np.column_stack([data, np.ones(data.shape[0])]) # Adding in a bias term
    samples,features = data.shape
    w = np.random.rand(features)*2 - np.ones(shape=features) #[0,1] -> [-100,100]
    for i in range(num_iters):
        index = np.random.randint(0, 1000)
        x = data[index]
        y = labels[index][0]
        loss = SGD(w, x, y, alpha=0.01)
        
    print(f'Average Loss: {avg_loss(data,w,labels)}, Accuracy on training set: {accuracy(data, labels, w)}')
    return w

feed_forward(data,labels)

Average Loss: 15.594220570847938, Accuracy on training set: 0.553


array([-4.63357755e-01,  4.83473841e-01,  6.41642174e-01,  2.47953082e-01,
       -5.00149450e-01,  3.82145940e-01, -9.80268205e-01, -1.98460028e-01,
        3.15161453e-01,  2.63474505e-01, -7.41164281e-01,  1.02922799e+00,
        3.60562450e-01, -1.15543365e-01, -2.97582850e-01, -3.28161236e-01,
       -5.28580780e-01,  4.82700019e-02,  5.77676766e-02, -1.64000280e-01,
        3.69621374e-01, -7.00390980e-01, -8.70787512e-02,  4.88203662e-01,
        2.27929968e-01,  1.24648484e-01, -3.01798492e-01,  6.65567725e-02,
        2.39701810e-01, -3.84236966e-01,  2.20433540e-01, -3.12710443e-01,
       -7.65901154e-01,  6.69138455e-01,  8.02368134e-01,  5.81369204e-01,
       -5.41712389e-01, -7.12778000e-01,  3.38324589e-01, -1.98735903e-01,
       -4.53238182e-01,  1.87480116e-02, -2.35351814e-01,  4.81750214e-01,
       -1.87884924e-01,  1.13864757e-01, -4.37835948e-01, -1.06380898e-01,
       -1.52991271e-01, -3.16967409e-01,  7.47753238e-01,  5.01508486e-01,
        5.98943760e-01,  

In [8]:
# Now let's play around with adding non-linear features from our input space
new_inp = add_polynomial_features(data,4)
feed_forward(new_inp,labels,num_iters=20000)

C:\Users\godha\AppData\Local\Temp\ipykernel_8080\3423659481.py:12: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-z)))


Average Loss: 15.308837450297187, Accuracy on training set: 0.5565


array([ 0.27771209,  0.29583632,  0.9654015 , ..., 25.0024772 ,
       -5.89584168,  0.22656735])

In [9]:
with_sin = np.column_stack([data,])
feed_forward(with_sin,labels,num_iters=20000)

Average Loss: 14.805225753014012, Accuracy on training set: 0.5765


array([-9.14748276e-01,  6.44526989e-02,  2.95723042e-01,  2.53244626e-01,
       -4.36409092e-01,  6.88848812e-01, -6.30831344e-01, -4.08070496e-01,
        3.18738722e-01, -8.23520955e-02, -7.21357356e-02, -3.37963597e-02,
       -2.25610876e-01, -1.23785701e-01,  2.55740623e-01, -1.33756264e-01,
       -1.56355180e-01,  8.05231597e-01,  1.92024766e-01,  9.70112145e-02,
        4.26015542e-01, -7.11353353e-01, -8.07115234e-02,  4.74016491e-01,
        2.28803954e-01, -6.41640722e-01,  6.70994620e-01,  1.32410084e-01,
       -3.74955089e-01,  8.13853293e-01,  2.24586440e-01,  4.49871348e-01,
       -8.37387094e-01, -6.09875610e-02,  2.05027071e-01, -2.93225620e-01,
       -5.57210238e-01, -5.13934246e-01,  3.62349788e-03, -1.21252343e-01,
       -9.47074781e-02,  5.16305531e-01,  4.87333028e-01,  3.32774504e-01,
        2.28689614e-01, -1.25098741e-01, -1.09073070e-01, -1.10800468e-01,
        1.06723139e-01, -3.52434117e-01,  6.48335860e-01,  2.12822074e-01,
       -3.66717519e-02, -

In [10]:
def compute_neighborhood_mean(data, window_size):
    image = data.reshape((20,20))
    padded_image = np.pad(image, pad_width=window_size//2, mode='constant')

    neighborhood_means = np.zeros_like(image, dtype=np.float64)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            neighborhood = padded_image[i:i+window_size, j:j+window_size]
            neighborhood_means[i, j] = np.mean(neighborhood)

    return neighborhood_means

def compute_neighborhood_mean2(image, window_size):
    image = image.reshape((20,20))

    padded_image = np.pad(image, pad_width=window_size//2, mode='constant')

    windowed_view = np.lib.stride_tricks.sliding_window_view(padded_image, (window_size, window_size))

    neighborhood_means = np.mean(windowed_view, axis=(-2, -1))

    return neighborhood_means



In [11]:
%%timeit
means = compute_neighborhood_mean2(data[0], 3)

95.6 µs ± 5.89 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%%timeit
means1 = compute_neighborhood_mean(data[0],3)

KeyboardInterrupt: 

In [ ]:
def add_neighborhood_feature(data, window_size=3):
    new_data = [] # store all the new data points in here
    for i in range(data.shape[0]): #i.e for each data point
        means = compute_neighborhood_mean2(data[i], window_size)
        new_data.append(np.concatenate((data[i], means.flatten())))
    return np.array(new_data)

print(data.shape)
new = add_neighborhood_feature(data)
print(new.shape)

        

(1000, 400)
(1000, 800)


In [ ]:
feed_forward(new, labels, num_iters=100000)

Average Loss: 11.073807434781434, Accuracy on training set: 0.693


array([-9.50794154e-02,  3.99009024e-01, -6.22428850e-01, -3.49684697e-01,
        6.38992160e-01,  1.17260052e+00,  6.69765954e-01,  1.68567141e-01,
        2.75288867e-02,  7.27609568e-02, -5.48760628e-01,  7.88568965e-01,
       -8.15581760e-01,  1.19165680e+00,  2.02778663e-01, -2.09777972e-01,
        4.76828078e-01, -1.65173568e+00, -1.81833646e-01,  8.32718407e-02,
        1.16052581e+00,  1.51576797e-01,  7.93021466e-01, -6.98457529e-02,
        1.39578732e+00, -1.35700489e+00, -1.08383600e+00, -1.84324652e-01,
        5.14390937e-01, -1.60303613e-02, -4.53977847e-01,  9.68031261e-01,
       -4.69193728e-01,  2.70655661e-01, -6.57587890e-01,  5.15507798e-01,
        8.92569220e-01, -5.25700852e-01, -2.14129387e-01,  1.34776020e-01,
       -9.64409769e-01,  1.19602723e+00,  1.85920978e-01, -9.88054099e-01,
       -6.91605737e-01,  2.53906668e-01,  7.47486662e-02, -9.02299729e-01,
        9.43133829e-01,  1.51399373e+00, -3.15583058e-01, -3.14029330e-01,
       -2.40342533e-01, -

In [ ]:
new2 = add_neighborhood_feature(data,window_size=6)
new3 = add_polynomial_features(new2, degree=3)

In [ ]:
weights = feed_forward(new3, labels, num_iters=1000000)

C:\Users\rohit\AppData\Local\Temp\ipykernel_23420\3423659481.py:12: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp(-z)))


Average Loss: 7.600665546630489, Accuracy on training set: 0.78


In [ ]:
task1_testdata = np.load("Task1_Testset500/data.npy")
task1_testlabels = np.load("Task1_Testset500/labels.npy")

print(f"accuracy on size 500 testing set: {accuracy(task1_testdata,task1_testlabels,weights)}")

ValueError: shapes (500,400) and (2524,) not aligned: 400 (dim 1) != 2524 (dim 0)